# Dataverse Interactive Notebook

This notebook demonstrates how you can use interactive notebooks with PowerPlatform .net core assembly

In [ ]:
#r "nuget:Microsoft.PowerPlatform.Dataverse.Client,*-*"
#r "nuget:Azure.Identity,*-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

using System;
using System.Linq;
using System.Collections;
using System.Collections.Generic;
using Azure.Core;
using Azure.Identity;
using Microsoft.PowerPlatform.Dataverse.Client;
using Microsoft.Xrm.Sdk;
using Microsoft.Xrm.Sdk.Metadata;
using Microsoft.Crm.Sdk.Messages;

The first step is to get the environment URL from the user. Visual Studio Code will prompt you for an environment URL when you run the next block. Provide an environment URL. Once the URL is provided, Azure Identity SDK can be used to get the token based on the current logged in user in VSCode. This user needs to have access to both Azure and Power Apps environment. DataverseClient will use Azure Identity SDK to retrieve the token. The client is now authenticated.

In [ ]:
var environment = await GetInputAsync("Enter Power Apps Environment URL. e.g. https://org.crm.dynamics.com");
var managedIdentity = new DefaultAzureCredential();
var client = new ServiceClient(tokenProviderFunction: async u =>
    (await managedIdentity.GetTokenAsync(
        new TokenRequestContext(new[] { $"{environment}/.default" }))).Token, 
        instanceUrl: new Uri(environment));

Since the `ServiceClient` is now authenticated, we can easily use this for all Dataverse calls.

In [ ]:
display($"Tenant: {client.OrganizationDetail.TenantId}\nOrganizationId: {client.OrganizationDetail.OrganizationId}\nFriendlyName: {client.OrganizationDetail.FriendlyName}");

Now let us try to get some metadata.

In [ ]:
var entities = client.GetEntityMetadata("systemuser", EntityFilters.Attributes);
display(entities.Attributes.Select(x=>new {x.ColumnNumber, x.LogicalName, DisplayName = x.DisplayName.UserLocalizedLabel?.Label ?? ""}).OrderBy(x=>x.ColumnNumber));

index,ColumnNumber,LogicalName,DisplayName
0,1,systemuserid,User
1,3,territoryid,Territory
2,4,organizationid,Organization
3,5,businessunitid,Business Unit
4,6,parentsystemuserid,Manager
5,7,firstname,First Name
6,8,salutation,Salutation
7,9,middlename,Middle Name
8,10,lastname,Last Name
9,11,personalemailaddress,Email 2


Now let us try some messages. For example, `RetrieveTotalRecordCountRequest` which gives us the table count (snapshot).

In [ ]:
var recordCountResponse = ((RetrieveTotalRecordCountResponse)client.Execute(new RetrieveTotalRecordCountRequest
{
    EntityNames = new string[] { "account", "contact", "systemuser" }
})).EntityRecordCountCollection;
display(recordCountResponse);

index,Key,Value
0,account,107
1,contact,63
2,systemuser,93


[PowerPlatform-DataverseServiceClient](https://github.com/microsoft/PowerPlatform-DataverseServiceClient) is still in preview. So, don't use it in production yet. But, it is great to use for data wrangling and some quick fixes to your Dataverse environment. I use it in my [Dataverse LINQPad Driver](https://github.com/rajyraman/Dataverse-LINQPad-Driver) as well.

Happy building.

[Natraj](https://twitter.com/rajyraman)